In [1]:
import pickle as pk
scomplex = pk.load(open(r"C:\Users\forre\Desktop\REU\TDA\Data\TDAOutputs\Pool26_2PCA100Cube20Perc_complex_withMax.p","rb"))

In [2]:
import numpy as np
import pandas as pd 

In [3]:
scomplex.keys()

dict_keys(['nodes', 'links', 'simplices', 'meta_data', 'meta_nodes', 'density', 'graph', 'maxima', 'states'])

In [4]:
states = scomplex['states']
print(states.keys())

dict_keys(['cube37_cluster0', 'cube289_cluster2', 'cube103_cluster0', 'cube545_cluster0', 'cube334_cluster0', 'cube257_cluster0', 'cube39_cluster0', 'cube180_cluster0', 'cube364_cluster0', 'cube23_cluster0', 'cube27_cluster0'])


In [5]:
df = pk.load(open(r"C:\Users\forre\Desktop\REU\TDA\Data\TDAOutputs\Pool26_df.p","rb"))

In [6]:
def getSubdf(scomplex, shape, df):
    """
    Returns the part of the data frame from the particular shape in the simplicial complex.
    params:
    scomplex: the entire simplicial complex
    shape: the particular shape being inspected (within the simplicial complex)
    df: the entire data frame
    
    Description:
    1. Get all the nodes from the particular simplicial complex. 
    2. Generate the indices we care about from the particular shape. To do this, we read each node and append it's 
    indices to a list. Then, we convert the list to a set and then back to a list to eliminate duplicates.
    3. Return the dataframe with only those indices.
    """
    nodes = scomplex.get('nodes')
    indices = []
    npShape = np.array(shape).flatten()
    for node in npShape:
        indices.append(nodes.get(node))
    indices = list(set([item for sublist in indices for item in sublist]))
    subdf = df.loc[indices]
    return subdf

In [38]:
subDf_dict = {}
summaryDf_dict = {}
variables = ["TN","TP","TEMP","DO","TURB","VEL","SS","WDP","CHLcal","SECCHI"]
for state in states:
    print("Max / state is: ", state)
    subDf_dict[state] = getSubdf(scomplex, states[state], df)
    summaryDf = pd.DataFrame()
    for var in variables:
        summaryDf[var] = subDf_dict[state][var].describe()
    summaryDf_dict[state] = summaryDf
    #display(summaryDf)
    

Max / state is:  cube37_cluster0
Max / state is:  cube289_cluster2
Max / state is:  cube103_cluster0
Max / state is:  cube545_cluster0
Max / state is:  cube334_cluster0
Max / state is:  cube257_cluster0
Max / state is:  cube39_cluster0
Max / state is:  cube180_cluster0
Max / state is:  cube364_cluster0
Max / state is:  cube23_cluster0
Max / state is:  cube27_cluster0


In [8]:
list(summaryDf_dict.keys())

['cube37_cluster0',
 'cube289_cluster2',
 'cube103_cluster0',
 'cube545_cluster0',
 'cube334_cluster0',
 'cube257_cluster0',
 'cube39_cluster0',
 'cube180_cluster0',
 'cube364_cluster0',
 'cube23_cluster0',
 'cube27_cluster0']

In [32]:
from scipy import stats
from statsmodels.stats import multitest
import math

In [26]:
keys = list(subDf_dict.keys())
for key in keys:
    print(key, "has a node count: " ,len(scomplex['states'][key]), " , and a datapoint count: ", summaryDf_dict[key].loc['count'][0])

cube37_cluster0 has a node count:  80  , and a datapoint count:  4151.0
cube289_cluster2 has a node count:  9  , and a datapoint count:  95.0
cube103_cluster0 has a node count:  60  , and a datapoint count:  1117.0
cube545_cluster0 has a node count:  2  , and a datapoint count:  24.0
cube334_cluster0 has a node count:  2  , and a datapoint count:  13.0
cube257_cluster0 has a node count:  4  , and a datapoint count:  34.0
cube39_cluster0 has a node count:  84  , and a datapoint count:  4532.0
cube180_cluster0 has a node count:  4  , and a datapoint count:  23.0
cube364_cluster0 has a node count:  44  , and a datapoint count:  747.0
cube23_cluster0 has a node count:  9  , and a datapoint count:  292.0
cube27_cluster0 has a node count:  8  , and a datapoint count:  88.0


One of the requirements for conducting a T test is that you have a normal distribution. However, if the sample size is large enough (here we use, $n \geq 100$), we can utilize the central limit theorem and have approximately a normal distribution. Thus, we only use the below for looking for potential states. (Reference: Applied Statistics for Engineers and Scientists, J. D. Petruccelli, B. Nandram, M. Chen, 2014, Previously published by Pearson)

In [46]:
keys = ['cube37_cluster0', 'cube103_cluster0', 'cube39_cluster0', 'cube364_cluster0', 'cube23_cluster0' ]
#keys = list(subDf_dict.keys()) # test everything
for var in variables:
    for state1 in keys:
        pvals = []
        for state2 in keys:
            if state1 == state2:
                continue
            print("var: ", var, "| state1: ", state1, ", datapoints: ", summaryDf_dict[state1].loc['count'][0], 
                  "| state2:", state2, ", datatpoints: ", summaryDf_dict[state2].loc['count'][0] )
            sample1 = subDf_dict[state1][var]
            sample2 = subDf_dict[state2][var]
            pvals.append(stats.ttest_ind(sample1, sample2, equal_var=False).pvalue)
            print("Sample means: ", sample1.mean(), " | ", sample2.mean())
            #print(stats.ttest_ind(sample1, sample2, equal_var=False))
        if False in multitest.multipletests(pvals, method = 'bonferroni')[0]:# 0 for reject nulls, 1 for adjusted p vals
            print(multitest.multipletests(pvals, method = 'bonferroni')[0])
        else:
            print("They are all different")
          
        print("---------")


var:  TN | state1:  cube37_cluster0 , datapoints:  4151.0 | state2: cube103_cluster0 , datatpoints:  1117.0
Sample means:  3.366257287400626  |  2.3368657117278424
var:  TN | state1:  cube37_cluster0 , datapoints:  4151.0 | state2: cube39_cluster0 , datatpoints:  4532.0
Sample means:  3.366257287400626  |  3.0290150044130626
var:  TN | state1:  cube37_cluster0 , datapoints:  4151.0 | state2: cube364_cluster0 , datatpoints:  747.0
Sample means:  3.366257287400626  |  4.08304953145917
var:  TN | state1:  cube37_cluster0 , datapoints:  4151.0 | state2: cube23_cluster0 , datatpoints:  292.0
Sample means:  3.366257287400626  |  2.9322294520547945
They are all different
---------
var:  TN | state1:  cube103_cluster0 , datapoints:  1117.0 | state2: cube37_cluster0 , datatpoints:  4151.0
Sample means:  2.3368657117278424  |  3.366257287400626
var:  TN | state1:  cube103_cluster0 , datapoints:  1117.0 | state2: cube39_cluster0 , datatpoints:  4532.0
Sample means:  2.3368657117278424  |  3.02901

### Preliminary Results from T test
The results from checking each variable against each one. If nothing is listed, then those were all found to be significantly different after doing a bonferroni correction. <br>
TN: the state from cube 39 and from cube 23 are not different and WDP: 39 and 23 are not different <br>
TEMP: Cube 39 and cube 364 are not different <br>
SS: 37 and 103 are not different <br>
From this, and our initial hypothesis about TN, TP, and SS being state variables, it seems that it is likely that 39 and 23 could be similar states, since they still have different TP and SS. The same idea applies to 37 and 103.

In [41]:
keys = ['cube39_cluster0', 'cube23_cluster0']
for var in variables:
    for state1 in keys:
        pvals = []
        for state2 in keys:
            if state1 == state2:
                continue
            print("var: ", var, "| state1: ", state1, "| state2:", state2 )
            sample1 = subDf_dict[state1][var]
            sample2 = subDf_dict[state2][var]
            pvals.append(stats.ttest_ind(sample1, sample2, equal_var=False).pvalue)
            print("Sample means: ", sample1.mean(), " | ", sample2.mean())
            #print(stats.ttest_ind(sample1, sample2, equal_var=False))
        if False in multitest.multipletests(pvals, method = 'bonferroni')[0]:# 0 for reject nulls, 1 for adjusted p vals
            print(multitest.multipletests(pvals, method = 'bonferroni')[0])
        else:
            print("They are all different")
          
        print("---------")


var:  TN | state1:  cube39_cluster0 | state2: cube23_cluster0
Sample means:  3.0290150044130626  |  2.9322294520547945
[False]
---------
var:  TN | state1:  cube23_cluster0 | state2: cube39_cluster0
Sample means:  2.9322294520547945  |  3.0290150044130626
[False]
---------
var:  TP | state1:  cube39_cluster0 | state2: cube23_cluster0
Sample means:  0.18745167696381287  |  0.12090753424657534
They are all different
---------
var:  TP | state1:  cube23_cluster0 | state2: cube39_cluster0
Sample means:  0.12090753424657534  |  0.18745167696381287
They are all different
---------
var:  TEMP | state1:  cube39_cluster0 | state2: cube23_cluster0
Sample means:  17.38018534863195  |  1.926027397260274
They are all different
---------
var:  TEMP | state1:  cube23_cluster0 | state2: cube39_cluster0
Sample means:  1.926027397260274  |  17.38018534863195
They are all different
---------
var:  DO | state1:  cube39_cluster0 | state2: cube23_cluster0
Sample means:  9.759951456310679  |  16.795205479452